# Edmonds-Karp Network Maximum Flow algorithm.
Edmonds-Karp algorithm is a specific implementation of the Ford-Fulkerson max. flow algorithm which utilises Breadth First Search to find shortest paths to the sink.

Ford-Fulkerson method does not specify how to actually find augmenting paths. This is there we can attemp to optimize it's perfomance.

The Edmonds-Karp algorithm uses a <font color='slate'><b>Breadth First Search</b></font> to find augmenting paths which yeilds an arguably <b>better time complexity</b> of <font color='orange'><b>O(VE^2)</b></font>.  

The major difference in this approach is that <u>the time complexity no longer depends on the capacity value of any edge</u>!


### Edmonds-Karp algorithm ensures Optimality of the Maximum Flow:
The Edmonds-Karp algorithm guarantees the optimality of the maximum flow it computes. When the algorithm terminates, the maximum flow obtained is indeed the maximum possible flow in the network. This optimality property ensures that the algorithm always provides the best solution in terms of maximizing the flow from the source to the sink.   

Optimality is ensured via the use of the Breath First Search.
### Use of Breadth First Search (BFS):
The Edmonds-Karp algorithm utilizes Breadth First Search as the main mechanism to find augmenting paths.

Here's how BFS is used in the algorithm:

- BFS starts from the source node and explores the graph in a breadth-first manner, visiting nodes in order of their distance from the source.
- BFS guarantees that the first augmenting path found is the shortest path from the sourse to the sink in terms of the number of edges.
- By using BFS, algorithm ensures that it explores paths systematically, gradually increasing the flow along the shortest paths before considering longer paths. This helps in finding the maximum flow efficiently.


In [1]:
class edmond_karp_example:
    """
    Example implementation of the Edmonds-Karp algorithm for maximum flow.
    """

    class Edge:
        """
        Represents an edge in the graph with its capacity, flow, and residual edge.
        """

        def __init__(self, frm, to, capacity):
            """
            Initializes the edge with the source, destination, and capacity.

            Args:
                frm (int): The source node index.
                to (int): The destination node index.
                capacity (float): The capacity of the edge.
            """
            self.frm = frm
            self.to = to
            self.residual = None
            self.capacity = capacity
            self.flow = 0
        
        def is_residual(self):
            """
            Checks if the edge is a residual edge (capacity is zero).

            Returns:
                bool: True if the edge is a residual edge, False otherwise.
            """
            return self.capacity == 0
        
        def remaining_capacity(self):
            """
            Calculates the remaining capacity of the edge.

            Returns:
                float: The remaining capacity of the edge.
            """
            return self.capacity - self.flow
        
        def augment(self, bottleneck):
            """
            Increases the flow of the edge by the given bottleneck value
            and decreases the flow of the residual edge by the same value.

            Args:
                bottleneck (float): The bottleneck value to augment the flow.
            """
            self.flow += bottleneck
            self.residual.flow -= bottleneck
     
        def __repr__(self):
            """
            Returns a string representation of the edge.

            Returns:
                str: A string representation of the edge.
            """
            return str((self.frm, self.to, self.capacity))
    
    class networkflow_solver_base:
        """
        Base class for a network maximum flow solver.
        """

        def __init__(self, n, s, t):
            """
            Initializes the solver with the number of nodes, source node index (s),
            and sink node index (t).

            Args:
                n (int): The number of nodes in the graph.
                s (int): The source node index.
                t (int): The sink node index.
            """
            self.n = n
            self.s = s
            self.t = t
            self.visited_token = 1
            self.visited_nodes = [0] * self.n
            self.solved = False
            self.max_flow = 0
            self.initialize_empty_flow_graph()
            
        def add_edge(self, frm, to, capacity):
            """
            Adds an edge to the graph with the given source, destination, and capacity.

            Args:
                frm (int): The source node index.
                to (int): The destination node index.
                capacity (float): The capacity of the edge.
            """
            if capacity <= 0:
                raise ValueError(f'Illegal capacity for forward edge: {capacity} <= 0')
            e1 = edmond_karp_example.Edge(frm, to, capacity)
            e2 = edmond_karp_example.Edge(to, frm, 0)
            e1.residual = e2
            e2.residual = e1
            self.graph[frm].append(e1)
            self.graph[to].append(e2)
            
        def initialize_empty_flow_graph(self):
            """
            Initializes an empty graph with no edges.
            """
            self.graph = [[] for _ in range(self.n)]
        
        def get_graph(self):
            """
            Returns the flow graph representation.

            Returns:
                list: The flow graph representation.
            """
            self.execute()
            return self.graph
    
        def get_max_flow(self):
            """
            Returns the maximum flow value.

            Returns:
                float: The maximum flow value.
            """
            self.execute()
            return self.max_flow
        
        def execute(self):
            """
            Executes the flow solver if it hasn't been solved before.
            """
            if self.solved:
                return
            self.solved = True
            self.solve()
            
        def solve(self):
            """
            Abstract method to be implemented by subclasses.
            """
            pass
        
        def visit(self, node):
            """
            Marks the node as visited.

            Args:
                node (int): The node to mark as visited.
            """
            self.visited_nodes[node] = self.visited_token
        
        def is_visited(self, node):
            """
            Checks if the node is visited.

            Args:
                node (int): The node to check.

            Returns:
                bool: True if the node is visited, False otherwise.
            """
            return self.visited_nodes[node] == self.visited_token
        
        def mark_all_nodes_unvisited(self):
            """
            Marks all nodes as unvisited.
            """
            self.visited_token += 1

    class edmonds_karp_solver(networkflow_solver_base):
        """
        Subclass of networkflow_solver_base that implements the Edmonds-Karp algorithm using BFS.
        """

        class Queue:
            """
            Simple queue class implemented using a Python list.
            """

            def __init__(self):
                self.items = []

            def is_empty(self):
                """
                Checks if the queue is empty.

                Returns:
                    bool: True if the queue is empty, False otherwise.
                """
                return len(self.items) == 0

            def enqueue(self, item):
                """
                Adds an item to the end of the queue.

                Args:
                    item: The item to enqueue.
                """
                self.items.append(item)

            def dequeue(self):
                """
                Removes and returns the first item from the queue.

                Returns:
                    Any: The first item in the queue.
                """
                if self.is_empty():
                    return print('Queue is empty')
                return self.items.pop(0)

            def print_queue(self):
                """
                Prints the items in the queue.
                """
                print(self.items)

        def visualize_flow(self):
            """
            Visualizes the network graph and its maximum flow.
            """
            self.execute()
            # Print the network graph with flow values
            print("Network Graph:")
            for node in range(self.n):
                edges = self.graph[node]
                for edge in edges:
                    print(f"({edge.frm}) -- [ {edge.flow}/{edge.capacity} ]--> ({edge.to}) | Is residual: {edge.is_residual()}")

            # Print the maximum flow value
            print("Maximum Flow:", self.max_flow)    
            
        def solve(self):
            """
            Solves the maximum flow problem using the Edmonds-Karp algorithm.
            """
            flow = float('inf')
            while flow != 0:
                self.mark_all_nodes_unvisited()
                flow = self.bfs()
                self.max_flow += flow

        def bfs(self):
            """
            Performs a breadth-first search (BFS) to find an augmenting path in the residual graph.

            Returns:
                float: The bottleneck value of the augmenting path.
            """
            queue = self.Queue()
            self.visit(self.s)
            queue.enqueue(self.s)
            self.prev = [None] * self.n
            
            while not queue.is_empty():
                node = queue.dequeue()
                
                # If we've reached the sink, break
                if node == self.t:
                    break 
                
                edges = self.graph[node]
                for edge in edges:
                    capacity = edge.remaining_capacity()
                    if capacity > 0 and not self.is_visited(edge.to):
                        self.visit(edge.to)
                        self.prev[edge.to] = edge
                        queue.enqueue(edge.to)
            
            # If sink is unreachable, return.
            if self.prev[self.t] is None:
                return 0
            
            bottleneck = float('inf')
            edge = self.prev[self.t]
            while edge is not None:
                bottleneck = min(bottleneck, edge.remaining_capacity())
                edge = self.prev[edge.frm]


            edge = self.prev[self.t]
            while edge is not None:
                edge.augment(bottleneck)
                edge = self.prev[edge.frm]
                
            return bottleneck


In [3]:
n = 12     # Number of nodes in the graph
s = n - 2  # Index of the 'source' node
t = n - 1  # Index of the 'sink' node

# Initializing solver class object
solver = edmond_karp_example.edmonds_karp_solver(n, s, t)

# Initializing edges from the source
solver.add_edge(solver.s, 0, 10)
solver.add_edge(solver.s, 1, 5)

solver.add_edge(solver.s, 2, 10)

# Initializing middle edges
solver.add_edge(0, 3, 10)
solver.add_edge(1, 2, 10)
solver.add_edge(2, 5, 15)
solver.add_edge(3, 1, 2)
solver.add_edge(3, 6, 15)
solver.add_edge(4, 1, 15)
solver.add_edge(4, 3, 3)
solver.add_edge(5, 4, 4)
solver.add_edge(5, 8, 10)
solver.add_edge(6, 7, 10)
solver.add_edge(7, 4, 10)
solver.add_edge(7, 5, 7)

# Initializing edges to the sink
solver.add_edge(6, solver.t, 15)
solver.add_edge(8, solver.t, 10)

print(f'Maximum flow sustainable on a graph: {solver.get_max_flow()}')

Maximum flow sustainable on a graph: 23


In [4]:
solver.visualize_flow()

Network Graph:
(0) -- [ -10/0 ]--> (10) | Is residual: True
(0) -- [ 10/10 ]--> (3) | Is residual: False
(1) -- [ -3/0 ]--> (10) | Is residual: True
(1) -- [ 3/10 ]--> (2) | Is residual: False
(1) -- [ 0/0 ]--> (3) | Is residual: True
(1) -- [ 0/0 ]--> (4) | Is residual: True
(2) -- [ -10/0 ]--> (10) | Is residual: True
(2) -- [ -3/0 ]--> (1) | Is residual: True
(2) -- [ 13/15 ]--> (5) | Is residual: False
(3) -- [ -10/0 ]--> (0) | Is residual: True
(3) -- [ 0/2 ]--> (1) | Is residual: False
(3) -- [ 13/15 ]--> (6) | Is residual: False
(3) -- [ -3/0 ]--> (4) | Is residual: True
(4) -- [ 0/15 ]--> (1) | Is residual: False
(4) -- [ 3/3 ]--> (3) | Is residual: False
(4) -- [ -3/0 ]--> (5) | Is residual: True
(4) -- [ 0/0 ]--> (7) | Is residual: True
(5) -- [ -13/0 ]--> (2) | Is residual: True
(5) -- [ 3/4 ]--> (4) | Is residual: False
(5) -- [ 10/10 ]--> (8) | Is residual: False
(5) -- [ 0/0 ]--> (7) | Is residual: True
(6) -- [ -13/0 ]--> (3) | Is residual: True
(6) -- [ 0/10 ]--> (7) | 